In [ ]:
%pip install --upgrade langchain langchain-experimental langchain-openai python-dotenv pyvis

In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()
# Get API key from environment variable 
# (make sure the key is present in .env file in the project directory)
api_key = os.getenv("OPENAI_API_KEY")

### LLM Graph Transformer
Using GPT-4o in all examples.

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

graph_transformer = LLMGraphTransformer(llm=llm)

### Extract graph data

In [ ]:
text = """
Luật số: 133/2025/QH15:
Điều 15. Sửa đổi, bổ sung, bãi bỏ một số điều, khoản của các luật có liên quan
1. Sửa đổi, bổ sung điểm e khoản 1 Điều 12 của Luật Quản lý và đầu tư vốn nhà nước tại doanh nghiệp số 68/2025/QH15 như sau:
“e) Doanh nghiệp sản xuất sản phẩm công nghệ cao, doanh nghiệp công nghệ cao, doanh nghiệp công nghệ chiến lược theo quy định của pháp luật về công nghệ cao, đầu tư lớn, tạo động lực phát triển nhanh cho các ngành, lĩnh vực khác và nền kinh tế;”.
2. Sửa đổi, bổ sung một số điều của Luật Khoa học, công nghệ và đổi mới sáng tạo số 93/2025/QH15 như sau:
a) Sửa đổi, bổ sung khoản 1 Điều 67 như sau:
“1. Sản phẩm công nghệ chiến lược thuộc Danh mục sản phẩm công nghệ chiến lược và sản phẩm công nghệ cao thuộc Danh mục sản phẩm công nghệ cao được khuyến khích phát triển được sản xuất bởi doanh nghiệp công nghệ chiến lược, doanh nghiệp công nghệ cao, doanh nghiệp sản xuất sản phẩm công nghệ cao; sản phẩm, hàng hóa từ kết quả của nhiệm vụ khoa học, công nghệ và đổi mới sáng tạo đặc biệt; sản phẩm, hàng hóa từ kết quả của nhiệm vụ khoa học, công nghệ và đổi mới sáng tạo trong nước được ưu đãi theo quy định của pháp luật về đấu thầu.”;
b) Bãi bỏ khoản 1 Điều 71.
"""

In [ ]:
documents = [Document(page_content=text)]
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [ ]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

#### Visualize graph

In [ ]:
from pyvis.network import Network

def visualize_graph(graph_documents):

    # Create network
    net = Network(height="1200px", width="100%", directed=True,
                      notebook=False, bgcolor="#222222", font_color="white")
    
    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}
    
    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])


    # Track which nodes are part of any relationship
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error

    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)
        
    output_file = "knowledge_graph.html"
    net.save_graph(output_file)
    print(f"Graph saved to {os.path.abspath(output_file)}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_file)}")
    except:
        print("Could not open browser automatically")
        
# Run the function
visualize_graph(graph_documents)

### Extract specific types of nodes

In [ ]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
graph_transformer_nodes_defined = LLMGraphTransformer(llm=llm, allowed_nodes=allowed_nodes)
graph_documents_nodes_defined = await graph_transformer_nodes_defined.aconvert_to_graph_documents(documents)

In [ ]:
print(f"Nodes:{graph_documents_nodes_defined[0].nodes}")
print(f"Relationships:{graph_documents_nodes_defined[0].relationships}")

### Extract specific types of relationships

In [ ]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
allowed_relationships = [
    ("Person", "WORKS_AT", "Organization"),
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Organization", "IN_LOCATION", "Location"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField")
]
graph_transformer_rel_defined = LLMGraphTransformer(
  llm=llm,
  allowed_nodes=allowed_nodes,
  allowed_relationships=allowed_relationships
)
graph_documents_rel_defined = await graph_transformer_rel_defined.aconvert_to_graph_documents(documents)

In [ ]:
# Visualize graph
visualize_graph(graph_documents_rel_defined)

### Extract Vietnamese legal entities with custom prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Custom prompt for Vietnamese legal document graph extraction.
VIETNAMESE_LEGAL_SYSTEM_PROMPT = (
    "You are an expert in Vietnamese legal document analysis. "
    "Your task is to extract a knowledge graph from Vietnamese legislative text.\n\n"
    "## Node types\n"
    "- **DOCUMENT**: A law or statute identified by its official number, "
    "e.g., 'Luật số 133/2025/QH15'.\n"
    "- **TERM**: A specific article, clause, or point that is being "
    "amended or repealed, described as its FULL reference including the parent law, "
    "e.g., 'điểm e khoản 1 Điều 12 của Luật Quản lý và đầu tư vốn nhà nước tại "
    "doanh nghiệp số 68/2025/QH15'.\n\n"
    "## Relationship types\n"
    "- **SUA_DO_BO_SUNG**: The DOCUMENT sửa đổi, bổ sung (amends/supplements) "
    "the TERM.\n"
    "- **BAI_BO**: The DOCUMENT bãi bỏ (repeals/removes) the TERM.\n\n"
    "## Extraction rules\n"
    "1. Create ONE DOCUMENT node for the issuing law (the one doing the amending), "
    "e.g., 'Luật số 133/2025/QH15'.\n"
    "2. For every 'Sửa đổi, bổ sung ...' action, create a TERM node whose "
    "ID is the FULL provision reference including its parent law number, then add a "
    "SUA_DO_BO_SUNG relationship from the DOCUMENT to that TERM.\n"
    "3. For every 'Bãi bỏ ...' action, create a TERM node whose ID is the "
    "FULL provision reference including its parent law number, then add a BAI_BO "
    "relationship from the DOCUMENT to that TERM.\n"
    "4. Do NOT create separate nodes for the full parent laws — only the targeted provisions.\n"
    "5. Use Vietnamese text for node IDs exactly as it appears in the source.\n\n"
    "Tip: The text below amends TWO provisions (items 1a and 2a) and repeals ONE "
    "(item 2b), so you should produce 1 DOCUMENT + 3 TERM nodes and "
    "3 relationships total."
)

custom_legal_prompt = ChatPromptTemplate.from_messages([
    ("system", VIETNAMESE_LEGAL_SYSTEM_PROMPT),
    ("human", "{input}"),
])

# Build transformer with domain constraints + custom prompt
allowed_nodes_legal = ["DOCUMENT", "TERM"]
allowed_relationships_legal = [
    ("DOCUMENT", "SUA_DO_BO_SUNG", "TERM"),
    ("DOCUMENT", "BAI_BO", "TERM"),
]

graph_transformer_legal = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=allowed_nodes_legal,
    allowed_relationships=allowed_relationships_legal,
    prompt=custom_legal_prompt,
    strict_mode=False,
)

graph_documents_legal = await graph_transformer_legal.aconvert_to_graph_documents(documents)
print(f"Nodes: {graph_documents_legal[0].nodes}")
print(f"\nRelationships: {graph_documents_legal[0].relationships}")

In [ ]:
def normalize_node_casing(graph_docs):
    """
    LLMGraphTransformer title-cases node IDs internally.
    This function normalizes all node IDs to lowercase.
    """
    for graph_doc in graph_docs:
        for node in graph_doc.nodes:
            node.id = node.id.lower()
        for rel in graph_doc.relationships:
            rel.source.id = rel.source.id.lower()
            rel.target.id = rel.target.id.lower()
    return graph_docs


graph_documents_legal = normalize_node_casing(graph_documents_legal)
print(f"Nodes: {graph_documents_legal[0].nodes}")
print(f"\nRelationships: {graph_documents_legal[0].relationships}")

In [ ]:
# Visualize the legal knowledge graph
visualize_graph(graph_documents_legal)